# Placement Tokens Demo

This notebook demonstrates placing a job on an AP, using a Placement Token for authorization.

Use Shift+Enter to run the cells in this notebook.

Run the following cell to perform some initial setup for the notebook.  A button will be created that will allow you to request the Placement Token.  You will use your NetID to request the token.

In [ ]:
import demo
import htcondor2
demo.setup()

After uploading the Placement Token, run the following cell to create the object named `schedd` with which you will interact with the Access Point.

In [ ]:
collector = htcondor2.Collector()
schedd_ad = collector.locate(
    htcondor2.DaemonType.Schedd, htcondor2.param["SCHEDD_HOST"]
)
schedd = htcondor2.Schedd(schedd_ad)

Verify that the connection to the Access Point is functional by querying for jobs.
If the following command does not show any errors, the connection is functional.
(There may already be some jobs placed on the AP from past sessions or by other users.)

In [ ]:
print("There are %d jobs currently placed at the AP." % len(schedd.query(projection=["ClusterId", "ProcId"])))

## Placing a job

Create the `submit` object, which will hold the submit description of a test job that you will place.
This test job will sleep for 30 seconds then add a message with the date.

In [ ]:
submit = htcondor2.Submit(
"""
Output = test_job_result.txt
Log = test_job.$(Cluster).log

Executable = /bin/bash
Arguments = "-c 'sleep 30; echo Job succeeded at `date`' 2>&1"
Transfer_Executable = false

Queue
"""
)

Place the job. The resulting job ID will be printed.

In [ ]:
placement = schedd.submit(submit, spool=True)
schedd.spool(placement)
print(f"Your placement has job ID {placement.cluster()}.0")

Check the status of the job

In [ ]:
demo.print_placement_status(placement, schedd)

If the job is completed, run the following to copy the results back to the current directory.  The file `test_job_result.txt` will be created.  If not, wait for a short amount of time and run the previous command again.

In [ ]:
schedd.retrieve(f"ClusterId == {placement.cluster()}")

This will print the result of the job.

In [ ]:
print(open("test_job_result.txt", "rt").read())